# Import libraries 

In [ ]:
import geotrellis.raster._
import geotrellis.raster.vectorize._
import geotrellis.raster.vectorize
import geotrellis.raster.io.geotiff.GeoTiff

import geotrellis.spark._
import geotrellis.spark.io.hadoop
import geotrellis.spark.io.hadoop._
import geotrellis.spark.io.RasterReader
import geotrellis.spark.tiling.FloatingLayoutScheme

import geotrellis.vector._

import org.apache.hadoop.fs.Path

import org.apache.spark.SparkContext
import org.apache.spark.rdd._
import org.apache.spark.mllib.linalg.{Vector, Vectors}

import java.net.URI
import scala.math.BigDecimal.RoundingMode

import geotrellis.raster._
import geotrellis.raster.vectorize._
import geotrellis.raster.vectorize
import geotrellis.raster.io.geotiff.GeoTiff
import geotrellis.spark._
import geotrellis.spark.io.hadoop
import geotrellis.spark.io.hadoop._
import geotrellis.spark.io.RasterReader
import geotrellis.spark.tiling.FloatingLayoutScheme
import geotrellis.vector._
import org.apache.hadoop.fs.Path
import org.apache.spark.SparkContext
import org.apache.spark.rdd._
import org.apache.spark.mllib.linalg.{Vector, Vectors}
import java.net.URI
import scala.math.BigDecimal.RoundingMode


# Important variables to make the library work

In [ ]:
implicit val sparkContext = sc
val rr = implicitly[RasterReader[HadoopGeoTiffRDD.Options, (ProjectedExtent, Tile)]]

sparkContext: org.apache.spark.SparkContext = org.apache.spark.SparkContext@7625b0a4
rr: geotrellis.spark.io.RasterReader[geotrellis.spark.io.hadoop.HadoopGeoTiffRDD.Options,(geotrellis.vector.ProjectedExtent, geotrellis.raster.Tile)] = geotrellis.spark.io.RasterReader$$anon$1@25af46f2


In [ ]:
val options =
HadoopGeoTiffRDD.Options(
  numPartitions = Some(100)
)

options: geotrellis.spark.io.hadoop.HadoopGeoTiffRDD.Options = Options(List(.tif, .TIF, .tiff, .TIFF),None,TIFFTAG_DATETIME,yyyy:MM:dd HH:mm:ss,Some(256),Some(100),Some(134217728),None)


In [ ]:
type LandsatKey = (ProjectedExtent, URI, Int)
// For each RDD, we're going to include more information in the key, including:
// - the ProjectedExtent
// - the URI
// - the future band value
def uriToKey(bandIndex: Int): (URI, ProjectedExtent) => LandsatKey =
  { (uri, pe) =>
    (pe, uri, bandIndex)
  }

defined type alias LandsatKey
uriToKey: (bandIndex: Int)(java.net.URI, geotrellis.vector.ProjectedExtent) => LandsatKey


# Parameters

In [ ]:
val HdfsUrl = "hdfs://hupi-factory-02-01-01-01/"
val dataRepo = "user/factory02/thailand_workshop/data_usgs/"
val saveRepo = "user/factory02/thailand_workshop/data_multiBands/"
val landsatName = "LC08_L1TP_125052_20171231_20180103_01_T1"

HdfsUrl: String = hdfs://hupi-factory-02-01-01-01/
dataRepo: String = user/factory02/thailand_workshop/data_usgs/
saveRepo: String = user/factory02/thailand_workshop/data_multiBands/
landsatName: String = LC08_L1TP_125052_20171231_20180103_01_T1


In [ ]:
val savePath = new Path(HdfsUrl + saveRepo + landsatName + ".tif")
val savePath_reduced = new Path(HdfsUrl + saveRepo + landsatName + "_reduced.tif")

savePath: org.apache.hadoop.fs.Path = hdfs://hupi-factory-02-01-01-01/user/factory02/thailand_workshop/data_multiBands/LC08_L1TP_125052_20171231_20180103_01_T1.tif
savePath_reduced: org.apache.hadoop.fs.Path = hdfs://hupi-factory-02-01-01-01/user/factory02/thailand_workshop/data_multiBands/LC08_L1TP_125052_20171231_20180103_01_T1_reduced.tif


# To avoid duplicates in HDFS

In [ ]:
val conf = sc.hadoopConfiguration 

// delete the images if it existed already
val fs = org.apache.hadoop.fs.FileSystem.get(new java.net.URI(HdfsUrl), conf)
fs.delete(savePath,true)
fs.delete(savePath_reduced,true)

conf: org.apache.hadoop.conf.Configuration = Configuration: core-default.xml, core-site.xml, mapred-default.xml, mapred-site.xml, yarn-default.xml, yarn-site.xml, hdfs-default.xml, hdfs-site.xml
fs: org.apache.hadoop.fs.FileSystem = DFS[DFSClient[clientName=DFSClient_NONMAPREDUCE_-1076060220_114, ugi=root (auth:SIMPLE)]]
res24: Boolean = false


false

In [ ]:
val conf = sc.hadoopConfiguration 

// delete the images if it existed already
val fs = org.apache.hadoop.fs.FileSystem.get(new java.net.URI(HdfsUrl), conf)
fs.delete(new Path(HdfsUrl + saveRepo),true)

conf: org.apache.hadoop.conf.Configuration = Configuration: core-default.xml, core-site.xml, mapred-default.xml, mapred-site.xml, yarn-default.xml, yarn-site.xml, hdfs-default.xml, hdfs-site.xml
fs: org.apache.hadoop.fs.FileSystem = DFS[DFSClient[clientName=DFSClient_NONMAPREDUCE_-715387487_123, ugi=root (auth:SIMPLE)]]
res5: Boolean = true


true

# Load GeoTiff from HDFS

In [ ]:
// Function to get GeoTiff for each bands
def bandPath(b: String) = s"hdfs://hupi-factory-02-01-01-01/user/factory02/thailand_workshop/data_usgs/${landsatName}/${landsatName}_${b}.TIF"

bandPath: (b: String)String


In [ ]:
// List of all bands needed from HDFS (except band 8 : Panchromatic?, it size is different than others, 
// the resolution is 15m while others are 30m -> maybe we can't treat this band the same ways like others ?) 
val listBands = Array("B1", "B2", "B3", "B4", "B5", "B6", "B7", "B9", "B10", "B11")

listBands: Array[String] = Array(B1, B2, B3, B4, B5, B6, B7, B9, B10, B11)


In [ ]:
// We initialize variable multiBands
var multibands = HadoopGeoTiffRDD[ProjectedExtent, LandsatKey, Tile](
      new Path(bandPath("B1")), 
      uriToKey(0),
      options)

// We create a loop that do the union of all bands 
for (i <- 1 to listBands.length - 1) {
  val band = listBands(i)
  val bandTile = HadoopGeoTiffRDD[ProjectedExtent, LandsatKey, Tile](
          new Path(bandPath(band)), 
          uriToKey(i),
          options)
  multibands = sc.union(multibands, bandTile) 
}

multibands: org.apache.spark.rdd.RDD[(LandsatKey, geotrellis.raster.Tile)] = UnionRDD[48] at union at <console>:153


# Combine all SinglebandTiles into one MultibandTile

In [ ]:
// Union these together, rearrange the elements so that we'll be able to group by key,
// group them by key, and the rearrange again to produce multiband tiles.
val sourceTiles: RDD[(ProjectedExtent, MultibandTile)] = {
 multibands
    .map { case ((pe, uri, bandIndex), tile) =>
      // Get the center of the tile, which we will join on
      val (x, y) = (pe.extent.center.x, pe.extent.center.y)

      // Round the center coordinates in case there's any floating point errors
      val center =
        (
          BigDecimal(x).setScale(5, RoundingMode.HALF_UP).doubleValue(),
          BigDecimal(y).setScale(5, RoundingMode.HALF_UP).doubleValue()
        )

      // Get the scene ID from the path
      val sceneId = uri.getPath.split('/').reverse.drop(1).head

      val newKey = (sceneId, center)
      val newValue = (pe, bandIndex, tile)
      (newKey, newValue)
    }
    .groupByKey()
    .map { case (oldKey, groupedValues) =>
      val projectedExtent = groupedValues.head._1
      val bands = Array.ofDim[Tile](groupedValues.size)
      for((_, bandIndex, tile) <- groupedValues) {
        bands(bandIndex) = tile
      }
      (projectedExtent, MultibandTile(bands))
    }
}

sourceTiles: org.apache.spark.rdd.RDD[(geotrellis.vector.ProjectedExtent, geotrellis.raster.MultibandTile)] = MapPartitionsRDD[51] at map at <console>:161


# Convert to a Raster

To be able to save in HDFS, we need to convert RDD[ProjectedExtent, Tile] to a Raster of MultibandTile

In [ ]:
val (_, metadata) = sourceTiles.collectMetadata[SpatialKey](FloatingLayoutScheme())
val tiles = sourceTiles.tileToLayout[SpatialKey](metadata)

metadata: geotrellis.spark.TileLayerMetadata[geotrellis.spark.SpatialKey] = TileLayerMetadata(uint16raw,GridExtent(Extent(528585.0, 1156335.0, 758985.0, 1394415.0),30.0,30.0),Extent(528585.0, 1162785.0, 755415.0, 1394415.0),EPSG:32648,KeyBounds(SpatialKey(0,0),SpatialKey(29,30)))
tiles: org.apache.spark.rdd.RDD[(geotrellis.spark.SpatialKey, geotrellis.raster.MultibandTile)] = ShuffledRDD[54] at reduceByKey at TileRDDMerge.scala:51


In [ ]:
// Raster original without compression
val raster = ContextRDD(tiles, metadata).stitch

In [ ]:
// Raster with resolution reduced
val raster_reduced =
  ContextRDD(tiles, metadata)
    .withContext { rdd =>
      rdd.mapValues { tile =>
        // Magic numbers! These were created by fiddling around with
        // numbers until some example landsat images looked good enough
        // to put on a map for some other project.
        val (min, max) = (4000, 15176)
        def clamp(z: Int) = {
          if(isData(z)) { if(z > max) { max } else if(z < min) { min } else { z } }
          else { z }
        }
        // we normalize the value of these rasters from min to 0, from max to 255 to reduce the resolution
        val blue = tile.band(0).map(clamp _).delayedConversion(ByteCellType).normalize(min, max, 0, 255)
        val green = tile.band(1).map(clamp _).delayedConversion(ByteCellType).normalize(min, max, 0, 255)
        val red = tile.band(2).map(clamp _).delayedConversion(ByteCellType).normalize(min, max, 0, 255)
        MultibandTile(blue, green, red)
      }
    }
    .stitch

raster_reduced: geotrellis.raster.Raster[geotrellis.raster.MultibandTile] = Raster(geotrellis.raster.ArrayMultibandTile@279b9eea,Extent(528585.0, 1156335.0, 758985.0, 1394415.0))


### We can directly save to HDFS

If we don't reduce the size, we have to set the configuration "spark.driver.maxResultSize": "2G" in customSparkConf in Edit metadata notebook.

For Kmeans, we need the original one, without compression

In [ ]:
GeoTiff(raster, metadata.crs).write(savePath)

No codec found for hdfs://hupi-factory-02-01-01-01/user/factory02/thailand_workshop/data_multiBands/LC08_L1TP_125052_20171231_20180103_01_T1.tif, writing without compression.


### Or save the reduced one

In [ ]:
GeoTiff(raster_reduced, metadata.crs).write(savePath_reduced)

No codec found for hdfs://hupi-factory-02-01-01-01/user/factory02/thailand_workshop/data_multiBands/LC08_L1TP_125052_20171231_20180103_01_T1_reduced.tif, writing without compression.
